# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [9]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [10]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 13, 2083, 56, 23, 14, 356, 1010, 201, 5, 17, 6, 506, 210, 1192, 12, 6, 1694, 112, 1668, 8, 789, 56, 522, 23, 2710, 2, 8, 106, 12, 2, 9, 131, 2, 796, 1753, 11, 796, 1129, 10, 10, 4, 201, 2, 4, 2474, 7, 4, 2, 223, 37, 412, 23, 6, 2, 751, 4566, 543, 2, 187, 4, 3286, 325, 999, 68, 2, 446, 4, 2, 9, 521, 5, 2, 34, 4, 2, 336, 1022, 5, 27, 2, 289, 3197, 1867, 2, 5, 117, 914, 134, 289, 1094, 28, 275, 2, 32, 7, 937, 28, 2116, 245, 153, 908, 10, 10, 4, 2, 26, 6, 254, 780, 2, 5, 2, 223, 545, 4973, 11, 148, 531, 4, 2, 9, 1058, 38, 4471, 94, 2, 3443, 6, 176, 7, 2, 3053, 82, 2, 245, 26, 400, 1670, 11, 661, 8, 818, 42, 2225, 2, 5, 38, 2589, 2964, 8, 135, 171, 7, 134, 2, 1345, 2, 261, 2, 76, 7, 4, 2, 1708, 215, 30, 1084, 2, 68, 2, 39, 2, 42, 2, 533, 39, 2, 3683, 7, 68, 2, 5, 4244, 2577, 4, 2, 81, 6, 1257, 227, 7, 2, 68, 1865, 56, 11, 4, 945, 5, 141, 21, 64, 1258, 8, 516, 54, 2, 427, 1670, 36, 26, 574, 11, 998, 513, 2, 60, 4, 993, 113, 7, 4, 2, 3641, 10, 10, 31, 7, 4, 293, 2, 7, 4, 20

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [11]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'was', 'meanwhile', 'she', 'are', 'as', 'need', 'boys', 'original', 'to', 'movie', 'is', 'flick', 'point', 'sick', 'that', 'is', 'uncle', 'never', 'carry', 'in', 'feature', 'she', 'etc', 'are', 'mexico', 'and', 'in', 'character', 'that', 'and', 'it', 'these', 'and', 'sister', 'tim', 'this', 'sister', 'cat', 'i', 'i', 'of', 'original', 'and', 'of', 'roger', 'br', 'of', 'and', 'whole', 'like', 'person', 'are', 'is', 'and', 'nearly', 'stilted', 'myself', 'and', 'however', 'of', 'stunts', 'black', 'cop', 'were', 'and', 'oh', 'of', 'and', 'it', 'actress', 'to', 'and', 'who', 'of', 'and', 'help', 'unlike', 'to', 'be', 'and', 'job', 'fortune', 'sucks', 'and', 'to', 'over', 'acted', 'while', 'job', 'entirely', 'one', 'money', 'and', 'an', 'br', 'directing', 'one', 'sake', 'tv', 'actors', 'open', 'i', 'i', 'of', 'and', 'he', 'is', 'each', 'die', 'and', 'to', 'and', 'whole', 'fight', 'album', 'this', 'though', 'hour', 'of', 'and', 'it', 'filled', 'her', 'sits'

In [12]:
lenghts = []
for review in X_train:
    lenghts.append(len(review))
print(max(lenghts))
print(min(lenghts))

2494
11


Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

2494 and 11


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [13]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [14]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
model = Sequential()
model.add(Embedding(vocabulary_size, 50, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

310501

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [15]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [16]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 6
val_size = 12500

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_test[:val_size], y_test[:val_size]
X_test, y_test = X_test[val_size:], y_test[val_size:]

# TODO: Train your model
model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

Train on 25000 samples, validate on 12500 samples
Epoch 1/6
25000/25000 [==============================] - 631s 25ms/step - loss: 0.4997 - acc: 0.7406 - val_loss: 0.3571 - val_acc: 0.8462
Epoch 2/6
25000/25000 [==============================] - 617s 25ms/step - loss: 0.3127 - acc: 0.8725 - val_loss: 0.3164 - val_acc: 0.8754
Epoch 3/6
25000/25000 [==============================] - 622s 25ms/step - loss: 0.2491 - acc: 0.9020 - val_loss: 0.3079 - val_acc: 0.8821
Epoch 4/6
25000/25000 [==============================] - 620s 25ms/step - loss: 0.2274 - acc: 0.9115 - val_loss: 0.3261 - val_acc: 0.8759
Epoch 5/6
25000/25000 [==============================] - 620s 25ms/step - loss: 0.1864 - acc: 0.9296 - val_loss: 0.3538 - val_acc: 0.8740
Epoch 6/6
25000/25000 [==============================] - 615s 25ms/step - loss: 0.1589 - acc: 0.9415 - val_loss: 0.3672 - val_acc: 0.8656


In [17]:
import os
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join("cache", model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

OSError: Unable to create file (unable to open file: name = 'cache/rnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [18]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.863920000038


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

Even with the minimal parameteres and hyperparameters LSTM outperformed both regular ML classifiers. 86% accuracy on test set versus 72% for BoW GaussianNB classifier and 82% for GradientBoostingClassifier.
Averege number of epochs is 20 - 40, also LSTM layer nodes and Embedding dimentions could have been much higher (up to 512).

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!